In [1]:
import numpy as np
import pandas as pd

In [40]:
data = np.random.normal(loc=0, scale=1, size=(1000, 10))

In [41]:
_a = np.random.normal(loc=0.3, scale=0.1, size=(1000, 1)).flatten()
_b = np.random.normal(loc=0.4, scale=0.1, size=(1000, 1)).flatten()
_c1 = np.random.normal(loc=0.5, scale=0.1, size=(1000, 1)).flatten()
_c2 = np.random.normal(loc=0.4, scale=0.2, size=(1000, 1)).flatten()
_d1 = np.random.normal(loc=0.3, scale=0.1, size=(1000, 1)).flatten()
_d2 = np.random.normal(loc=0.2, scale=0.05, size=(1000, 1)).flatten()

In [43]:
a = data[:, 0] > _a
b = data[:, 1] < _b
c = np.minimum(data[:, 2] > _c1, data[:, 3] > _c2)
d = np.minimum(data[:, 4] > _d1, data[:, 5] < _d2)

In [44]:
a.sum(), b.sum(), c.sum(), d.sum()

(np.int64(398), np.int64(655), np.int64(111), np.int64(237))